In [ ]:
import pandas as pd
import numpy as np
import altair as alt

## Setup
* Use the two isolated points (economic optimisation and environmental optimisation) to forecast for both perspectives
* Statistiacal analysis of grid signals (price & CO_2eq) to determine variance, covariance and correlation
* Test the forecast for both environmental and economic performance

In [ ]:
price = pd.read_csv('Data/PriceCurve_SE3_2021.csv', sep = ';')
co2_pro = pd.read_csv('Data/production_emissions.csv')
co2_con = pd.read_csv('Data/consumption_emissions.csv')
pv = pd.read_csv('Data/pv_sam.csv')
load = pd.read_csv('Data/LoadCurve.csv', sep = ';')

data = load
data['Price'] = price['Grid_Price']
data['CO_2_eq'] = co2_pro['carbon_intensity_production_avg']
data['solar_PV'] = pv

#Converting from MW to kW
data['Load']= (data['Load'] * 1000)
#solar data is already in kW, but needs to be multiplied by the scaling factor
data['solar_PV']= (data['solar_PV'])
data['Price']= (data['Price'] / 1000)
data['CO_2_eq']= (data['CO_2_eq'] / 1000)

data['Hour']= (data['Hour']).astype('int')

#data.head(48)
#data.head(10)

In [ ]:
## Primary data parameters of our scenarios
pv_price= 80                #https://data.nrel.gov/submissions/53 in EUR/kW
bess_price= 200             #https://doi.org/10.1016/j.solener.2018.08.061 in EUR/kWh, adjusted for price decreases
bess_bos= 250               #https://www.energy-storage.news/li-ion-bess-costs-could-fall-47-by-2030-nrel-says-in-long-term-forecast-update/ in EUR/kW
## Also see: https://www.nrel.gov/grid/assets/pdfs/second_grid_sim_zagoras.pdf
# pv_opex= 17                 #EUR/kWh ->reference in excel
# bess_opex= 0.125            #EUR/kWh ->reference in excel
pv_opex= 3
bess_opex= 6

pv_co2=             33      #kgCO2eq/kW_powerDC ->reference in excel
bess_co2=           100     #kgCO2eq/kWh_capacity ->reference in excel
pv_opex_co2=        0       #kgCO2eq/kW_powerDC ->assumption
bess_opex_co2=      0       #kgCO2eq/kW_powerDC ->assumption
discount_rate=      0.0485  #assumption
degradation_rate=   0.025   #assumption (based on reaching 80% SoH in 8 years)
lifetime_project=   32      #for the project lifetime
lifetime_bess=      8       #for the BESS lifetime
eol_rate=           30

params = {
    'pv_price':         pv_price,
    'bess_price':       bess_price,
    'bess_bos':         bess_bos,
    'pv_opex':          pv_opex,
    'bess_opex':        bess_opex,
    'pv_co2':           pv_co2,
    'bess_co2':         bess_co2,
    'pv_opex_co2':      pv_opex_co2,
    'bess_opex_co2':    bess_opex_co2,
    'discount_rate':    discount_rate,
    'lifetime_project': lifetime_project,
    'lifetime_bess':    lifetime_bess,
    'degradation_rate': degradation_rate,
    'eol_rate':         eol_rate
}

In [ ]:
results_caseb_econ= (pd.read_csv(
    'Results/Batch5/CaseB/results_econ.csv',
    sep= ';',
))
results_caseb_env= (pd.read_csv(
    'Results/Batch5/CaseB/results_env.csv',
    sep= ';',
))

In [ ]:
econ_point= results_caseb_econ[
    (results_caseb_econ['P_PV'] == 39)
    & (results_caseb_econ['P_BESS'] == 6)
    & (results_caseb_econ['E_BESS'] == 18)
    & (results_caseb_econ['T_BESS'] == 3)
    #& (results_caseb_econ['Pareto Curve'] == 'Case B: Econ. Invstr')
]

econ_point= econ_point.reset_index(drop= True)
#econ_point['label']= '0 %'
#econ_point['label']
econ_point['capex_bess']
#econ_point.info()

In [ ]:
env_point= results_caseb_env[
    (results_caseb_env['P_PV'] == 39)
    & (results_caseb_env['P_BESS'] == 21)
    & (results_caseb_env['E_BESS'] == 21)
    & (results_caseb_env['T_BESS'] == 1)
    #& (results_caseb_env['Pareto Curve'] == 'Case B: Env. Invstr')
]

env_point= env_point.reset_index(drop= True)
#env_point['label']= '0 %'
#env_point['label']
env_point['capex_bess']
#env_point.info()

___

In [ ]:
print(f'Price Variance: {data.Price.var()}')
print('-'* 50)
print(f'Price Standard Deviation: {data.Price.std()}')
print('-'* 50)
print(f'CO2eq Variance: {data.CO_2_eq.var()}')
print('-'* 50)
print(f'CO2eq Standard Deviation: {data.CO_2_eq.std()}')
print('-'* 50)
print(f'Price and CO2eq Covariance: {np.cov(data.Price, data.CO_2_eq)[0][1]}')
print('-'* 50)
print(f'Price and CO2eq Correlation: {data.Price.corr(data.CO_2_eq)}')

In [ ]:
## Points to use
econ_point.info()
env_point.info()

In [ ]:
grid_ef= data.CO_2_eq.mean()                            # kg CO_2eq/MWh_produced
production_impact_price_system= econ_point['capex_co2'] # kg CO_2eq/kWh_capacity
production_impact_co2eq_system= env_point['capex_co2']  # kg CO_2eq/kWh_capacity
capex_price_system= econ_point['capex_co2']             # kg CO_2eq/kWh_capacity
capex_co2eq_system= env_point['capex_co2']              # kg CO_2eq/kWh_capacity
savings_price_system= econ_point['savings']             # Check Units!
savings_co2eq_system= env_point['savings']              # Check Units!
pvbess_footprint= 11 * 1/0.95                           # kg CO_2eq/MWh_produced
evolution_factor= (
    1
    -
    1/1000000   # Efficiencies and new resources that lower the footprint
    +
    0/100     # Degradations in the network and other mechanisms that increase the footprint
)
#degradation_factor = 1.000005 # Rudimentary (for now) factor that accounts for BESS degradation over time.
degradation_factor = 1 + params['degradation_rate']

In [ ]:
energy_produced_econ = np.array(range(32))
energy_produced_env = np.array(range(32))
for i in range(1, len(energy_produced_econ)):
    energy_produced_econ[i] = i * (econ_point['pv2load'] + econ_point['pv2bess']).round()
    energy_produced_env[i] = i * (env_point['pv2load'] + env_point['pv2bess']).round()
data_econ = pd.DataFrame(energy_produced_econ, columns= ['MWh_produced'])
data_econ['Year'] = 0
data_env = pd.DataFrame(energy_produced_env, columns= ['MWh_produced'])
data_env['Year'] = 0
for i in range (1, 32):
    data_econ['Year'][i] = i
    data_env['Year'][i] = i
data_econ

In [ ]:
columns= [
    'P_PV',
    'P_BESS',
    'E_BESS',
    'lcoe_investor',
    'lcoe_enduser',
    'lco2_investor',
    'lco2_enduser',
    'capex',
    'capex_pv',
    'capex_bess',
    'capex_co2',
    'capex_co2_pv',
    'capex_co2_bess',
    'savings',
    'energy_procurement_costs',
    'energy_procurement_costs_load',
    'energy_procurement_costs_bess',
    'co2avoided',
    'co2burden',
    'co2burden_load',
    'co2burden_bess',
    'net_savings',
    'net_co2avoided',
    'profit',
    'profit_pv',
    'profit_bess',
    'co2abatement',
    'co2abatement_pv',
    'co2abatement_bess'
]


print('Economic Point')
for column in columns:
    print(f'{column}: {econ_point[column].values}')

In [ ]:
columns= [
    'P_PV',
    'P_BESS',
    'E_BESS',
    'lcoe_investor',
    'lcoe_enduser',
    'lco2_investor',
    'lco2_enduser',
    'capex',
    'capex_pv',
    'capex_bess',
    'capex_co2',
    'capex_co2_pv',
    'capex_co2_bess',
    'savings',
    'energy_procurement_costs',
    'energy_procurement_costs_load',
    'energy_procurement_costs_bess',
    'co2avoided',
    'co2burden',
    'co2burden_load',
    'co2burden_bess',
    'net_savings',
    'net_co2avoided',
    'profit',
    'profit_pv',
    'profit_bess',
    'co2abatement',
    'co2abatement_pv',
    'co2abatement_bess'
]

print('Environmental Point')
for column in columns:
    print(f'{column}: {env_point[column].values}')

In [ ]:
energy_produced_econ = np.array(range(32))
energy_produced_env = np.array(range(32))
for i in range(1, len(energy_produced_econ)):
    energy_produced_econ[i] = i * (econ_point['pv2load'] + econ_point['grid2bess']).round()
    energy_produced_env[i] = i * (env_point['pv2load'] + env_point['grid2bess']).round()
data_econ_econ = pd.DataFrame(energy_produced_econ, columns= ['MWh_produced'])
data_econ_econ['Year'] = 0
data_econ_env = pd.DataFrame(energy_produced_env, columns= ['MWh_produced'])
data_econ_env['Year'] = 0
for i in range (1, 32):
    data_econ_econ['Year'][i] = i
    data_econ_env['Year'][i] = i

data_econ['grid_footprint_relic'] = 0
# data_econ['weo_sps'] = 0
# data_econ['weo_aps'] = 0
# data_econ['weo_ntz'] = 0
data_econ['price_optimised_system'] = production_impact_price_system
data_econ['co2eq_optimised_system'] = production_impact_co2eq_system
data_econ['statusquo_savings'] = 0
data_econ['Economic_Optimal'] = -econ_point['capex']
data_econ['Environmental_Optimal'] = -env_point['capex']


for i in range(1, len(energy_produced_econ)):
    if i in [8, 16, 24, 32]:
    #     data_econ['grid_footprint_relic'][i] = grid_footprint_relic
    #     data_econ['pvbess_footprint_min19'][i] = production_impact_min19
    #     data_econ['pvbess_footprint_min22'][i] = production_impact_min22
    #     data_econ['pvbess_footprint_max19'][i] = production_impact_max19
    #     data_econ['pvbess_footprint_max22'][i] = production_impact_max22
    #     data_econ['pvbess_footprint_NV_target'][i] = production_impact_NV_target
        data_econ['grid_footprint_relic'].iloc[i] = int(data_econ['grid_footprint_relic'].iloc[i-1] + grid_ef * data.Load.sum() * (evolution_factor ** i/1))
        # data_econ['weo_sps'].iloc[i] = int(data_econ['weo_sps'].iloc[i-1] + grid_ef * (origin['pv2load'].iloc[27] + origin['grid2bess'].iloc[27]).round() * (1 + (weo_scenarios_extrapolated['ef_change[%]'].iloc[i])))
        # data_econ['weo_aps'].iloc[i] = int(data_econ['weo_aps'].iloc[i-1] + grid_ef * (origin['pv2load'].iloc[27] + origin['grid2bess'].iloc[27]).round() * (1 + (weo_scenarios_extrapolated['ef_change[%]'].iloc[i + 32])))
        # data_econ['weo_ntz'].iloc[i] = int(data_econ['weo_ntz'].iloc[i-1] + grid_ef * (origin['pv2load'].iloc[27] + origin['grid2bess'].iloc[27]).round() * (1 + (weo_scenarios_extrapolated['ef_change[%]'].iloc[i + 64])))
        data_econ.price_optimised_system.iloc[i] = data_econ.price_optimised_system.iloc[i-1] + econ_point['capex_co2_bess'] + (econ_point['co2burden']).round() * (degradation_factor ** i/1)
        data_econ.co2eq_optimised_system.iloc[i] = data_econ.co2eq_optimised_system.iloc[i-1] + env_point['capex_co2_bess'] + (env_point['co2burden']).round() * (degradation_factor ** i/1)
        data_econ.Economic_Optimal.iloc[i] = data_econ.Economic_Optimal.iloc[i-1] + (int((-econ_point['capex_bess'] + econ_point['savings']) / ((1 + params['discount_rate']) ** i)))
        data_econ.Environmental_Optimal.iloc[i] = data_econ.Environmental_Optimal.iloc[i-1] + (int((-env_point['capex_bess'] + env_point['savings']) / ((1 + params['discount_rate']) ** i)))
    else:
        data_econ['grid_footprint_relic'].iloc[i] = int(data_econ['grid_footprint_relic'].iloc[i-1] + grid_ef * data.Load.sum() * (evolution_factor ** i/1))
        # data_econ['weo_sps'].iloc[i] = int(data_econ['weo_sps'].iloc[i-1] + grid_ef * (origin['pv2load'].iloc[27] + origin['grid2bess'].iloc[27]).round() * (1 + (weo_scenarios_extrapolated['ef_change[%]'].iloc[i])))
        # data_econ['weo_aps'].iloc[i] = int(data_econ['weo_aps'].iloc[i-1] + grid_ef * (origin['pv2load'].iloc[27] + origin['grid2bess'].iloc[27]).round() * (1 + (weo_scenarios_extrapolated['ef_change[%]'].iloc[i + 32])))
        # data_econ['weo_ntz'].iloc[i] = int(data_econ['weo_ntz'].iloc[i-1] + grid_ef * (origin['pv2load'].iloc[27] + origin['grid2bess'].iloc[27]).round() * (1 + (weo_scenarios_extrapolated['ef_change[%]'].iloc[i + 64])))
        data_econ['price_optimised_system'].iloc[i] = int(data_econ['price_optimised_system'].iloc[i-1] + (econ_point['co2burden']).round() * (degradation_factor ** i/1))
        data_econ['co2eq_optimised_system'].iloc[i] = int(data_econ['co2eq_optimised_system'].iloc[i-1] + (env_point['co2burden']).round() * (degradation_factor ** i/1))
        data_econ['Economic_Optimal'].iloc[i] = int(data_econ['Economic_Optimal'].iloc[i-1] + econ_point['savings'] / ((1 + params['discount_rate']) ** i)) 
        data_econ['Environmental_Optimal'].iloc[i] = int(data_econ['Environmental_Optimal'].iloc[i-1] + env_point['savings'] / ((1 + params['discount_rate']) ** i))


## Divide by 1000000 to convert from grams to tonnes
data_econ.grid_footprint_relic = data_econ.grid_footprint_relic / 1000000
data_econ.price_optimised_system = data_econ.price_optimised_system / 1000000
data_econ.co2eq_optimised_system = data_econ.co2eq_optimised_system / 1000000
# data_econ.weo_sps = data_econ.weo_sps / 1000000
# data_econ.weo_aps = data_econ.weo_aps / 1000000
# data_econ.weo_ntz = data_econ.weo_ntz / 1000000
data_econ.Economic_Optimal = data_econ.Economic_Optimal / 1000000
data_econ.Environmental_Optimal = data_econ.Environmental_Optimal / 1000000

#data_econ = data_econ.drop(columns = data_econ.columns[0], axis = 1)
data_econ['delta_offsets'] = ((data_econ.grid_footprint_relic - data_econ.co2eq_optimised_system) / data_econ.grid_footprint_relic) * 100
#data_econ['delta_savings'] = ((data_econ.Economic_Optimal - data_econ.Environmental_Optimal) / data_econ.Economic_Optimal) * 100

for i in range(len(data_econ)):
    if data_econ['delta_offsets'][i] < 0:
        data_econ['delta_offsets'][i] = 0

data_econ

In [ ]:
data_econ.plot.line(
    y = [
    # 'weo_sps',
    # 'weo_aps',
    # 'weo_ntz',
    'grid_footprint_relic',
    'price_optimised_system',
    'co2eq_optimised_system',
    ],
    title = 'kg CO_2eq/MWh_delivered',
)

In [ ]:
alt.layer(
    alt.Chart(data_econ).mark_line(color= 'blue').encode(
        x= alt.X(
            'Year:Q',
            scale= alt.Scale(
                domain= (0, 32)
            )),
        y= alt.Y(
            'grid_footprint_relic:Q',
            axis= alt.Axis(
                title= 'Net Accumulated CO2eq Footprint [tCO2eq]'
            ))
    ),
    alt.Chart(data_econ).mark_line(color= 'red').encode(
        x= alt.X(
            'Year:Q',
            scale= alt.Scale(
                domain= (0, 32)
            )),
        y= alt.Y(
            'price_optimised_system:Q',
            axis= alt.Axis(
                title= 'Net Accumulated CO2eq Footprint [tCO2eq]'
            ))
    ),
    alt.Chart(data_econ).mark_line(color= 'green').encode(
        x= alt.X(
            'Year:Q',
            scale= alt.Scale(
                domain= (0, 32)
            )),
        y= alt.Y(
            'co2eq_optimised_system:Q',
            axis= alt.Axis(
                title= 'Net Accumulated CO2eq Footprint [tCO2eq]'
            ))
    )
).properties(
    height= 300,
    width= 900
)

In [ ]:
alt.layer(
    alt.Chart(data_econ).mark_line(color= 'blue').encode(
        x= alt.X(
            'Year:Q',
            scale= alt.Scale(
                domain= (0, 32)
            )),
        y= alt.Y(
            'statusquo_savings:Q',
            axis= alt.Axis(
                title= 'Net Savings [Mil. EUR]'
            ))
    ),
    alt.Chart(data_econ).mark_line(color= 'red').encode(
        x= alt.X(
            'Year:Q',
            scale= alt.Scale(
                domain= (0, 32)
            )),
        y= alt.Y(
            'Economic_Optimal:Q',
            axis= alt.Axis(
                title= 'Net Savings [Mil. EUR]'
            ))
    ),
    alt.Chart(data_econ).mark_line(color= 'green').encode(
        x= alt.X(
            'Year:Q',
            scale= alt.Scale(
                domain= (0, 32)
            )),
        y= alt.Y(
            'Environmental_Optimal:Q',
            axis= alt.Axis(
                title= 'Net Savings [Mil. EUR]'
            ))
    )
).properties(
    height= 300,
    width= 900
)